In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import numpy as np
from common import *
from PatientVec.Experiments.hyperparam_exps import *

In [8]:
import argparse
parser = argparse.ArgumentParser(description='Run Diagnosis experiments')
parser.add_argument("--data_dir", type=str, required=True)
parser.add_argument('--display', dest='display', action='store_true')
parser.add_argument("--output_dir", type=str)
parser.add_argument("--mock", dest='mock', action='store_true')

args = parser.parse_args(['--data_dir=.', '--output_dir=outputs/', '--display'])

In [20]:
from dataloaders import mortality_dataset, readmission_dataset, diagnosis_dataset
data = diagnosis_dataset(args) #, _type='30day')

INFO - 2019-03-20 15:28:18,509 - Reading Structured data ...
INFO - 2019-03-20 15:28:18,928 - Reading Notes ...
INFO - 2019-03-20 15:28:37,133 - Stratifying ...


In [21]:
train_data, dev_data = get_basic_data(data, structured=True, truncate=90)

Pos Percentage [0.21358209 0.07716941 0.10244004 0.31971179 0.13028647 0.13014724
 0.20870897 0.07160013 0.26850917 0.31939852 0.09485189 0.19033033
 0.28253681 0.41581677 0.26697762 0.07180897 0.13039089 0.0897003
 0.04953183 0.04076021 0.08917818 0.14152947 0.18249852 0.14615893
 0.07797   ]


INFO - 2019-03-20 15:28:52,919 - Maximum Sentence Length 721825.000000 , 90 percentile length 16714.000000 ... 
INFO - 2019-03-20 15:29:13,779 - Truncated all ...


Pos Percentage [0.20548768 0.06969758 0.10194272 0.31744442 0.12237132 0.13458842
 0.2032846  0.06649309 0.25776087 0.31484078 0.0929301  0.19587422
 0.29541358 0.42339275 0.25475666 0.06789505 0.12096936 0.08531945
 0.0530743  0.03745243 0.0817144  0.1327859  0.17684759 0.13198478
 0.07790907]


INFO - 2019-03-20 15:29:16,224 - Maximum Sentence Length 407062.000000 , 90 percentile length 16420.000000 ... 
INFO - 2019-03-20 15:29:19,148 - Truncated all ...


In [ ]:
config = {'vocab' : data.vocab, 'stop_words' : True, 'exp_name' : data.name, 
          'type' : data.metrics_type, 'norm' : 'l2', 'constant_mul' : 1.0, 'has_structured' : True}
lr = LR(config)
lr.train(train_data)
metrics = lr.evaluate(dev_data, save_results=True)


Normalising using l2
Using Norm from linalg
Multiplying by constant ,  1.0



Clipping ...
Normalising using l2
Using Norm from linalg
Multiplying by constant ,  1.0


Saving Models
==============

In [ ]:
from common import generate_latex_tables
keys_to_use = ['roc_auc', 'pr_auc']
generate_latex_tables(data, keys_to_use)

In [ ]:
dirname = 'outputs/Readmission/Basic/'
exps = os.listdir(dirname)
for e in sorted(exps) :
    if 'Structured' in e :
        print(e)
        print_results_from_model(get_latest_model(os.path.join(dirname, e)))

In [ ]:
dirname = 'outputs/Diagnosis/Basic/'
exps = os.listdir(dirname)
for e in sorted(exps) :
    if 'Structured' in e :
        print(e)
        print_results_from_model(get_latest_model(os.path.join(dirname, e)))

In [5]:
keys = ['roc_auc', 'pr_auc']
yr_df = defaultdict(list)
for yr in [1] :
    for model in ['baselines', 'Basic', 'Attention', 'Structured Attention'] :
        name = os.path.join('outputs/Mortality_30day/', model)
        dirs = os.listdir(name)
        dfs = {}
        for d in sorted(dirs) :
            dirname = os.path.join(name, d)
            model_name = get_latest_model(dirname)
            if model_name is not None :
                df = json.load(open(os.path.join(model_name, 'evaluate.json')))
                dfs[d] = {k:v for k, v in df.items() if k in keys}
        
        dfs = pd.DataFrame(dfs).transpose()
        yr_df[yr] += [dfs]
        
    yr_df[yr] = pd.concat(yr_df[yr], axis=0)
yr_df = pd.concat(yr_df.values(), axis=1, keys=yr_df.keys()).round(3)
yr_df.columns = yr_df.columns.swaplevel(0, 1)
yr_df.sort_index(axis=1, level=0, inplace=True)
display(HTML(yr_df.to_html()))

,pr_auc,roc_auc
,1,1
LR+BOW+norm=l1+clip=False,0.592,0.862
LR+BOW+norm=l2+clip=False,0.784,0.929
LR+BOW+norm=l2+clip=False+Structured,0.801,0.939
LR+BOW+norm=l3+clip=False,0.794,0.923
Average(hs=256),0.743,0.907
Average(hs=256)+Structured,0.651,0.897
"CNN(hs=64)(kernels=3,5,7,9)",0.789,0.928
"CNN(hs=64)(kernels=3,5,7,9)+Structured",0.782,0.927
LSTM(hs=128),0.802,0.923
